In [1]:
# Install dependencies
!pip install -q -U google-generativeai pypdf reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1

In [8]:
import os, re
import google.generativeai as genai
from pypdf import PdfReader
from reportlab.lib.pagesizes import LETTER
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

In [3]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GOOGLE_API_KEY")
except ImportError:
    api_key = os.environ.get("GOOGLE_API_KEY")

if not api_key:
    api_key = input("Enter your Google API Key: ")

genai.configure(api_key=api_key)
MODEL_NAME = "gemini-2.5-flash"

In [4]:
class SessionAgent:
    def __init__(self, name, system_instruction):
        self.name = name
        self.model = genai.GenerativeModel(
            model_name=MODEL_NAME,
            system_instruction=system_instruction
        )
        self.chat_session = self.model.start_chat(history=[])

    def send_message(self, message):
        print(f"\n🤖 {self.name} is thinking...")
        try:
            response = self.chat_session.send_message(message)
            return response.text
        except Exception as e:
            return f"Error: {e}"

In [9]:
# --- 3. PDF TOOLS (INPUT & OUTPUT) ---

def create_test_pdf(filename="vendor_contract.pdf"):
    """Creates a dummy PDF file on disk to test the ingestion."""
    from reportlab.pdfgen import canvas
    c = canvas.Canvas(filename)
    text = c.beginText(40, 750)
    text.setFont("Helvetica", 12)
    
    content = """
    VENDOR SERVICE AGREEMENT (DRAFT)
    
    1. PAYMENT TERMS: Client shall pay all undisputed invoices within Net 90 Days.
    2. LIABILITY: Vendor's total liability shall not exceed $100 USD.
    3. JURISDICTION: This agreement is governed by the laws of Antarctica.
    4. TERMINATION: Vendor may terminate immediately without notice.
    """
    
    for line in content.split("\n"):
        text.textLine(line)
    c.drawText(text)
    c.save()
    print(f"✅ Created test file: {filename}")
    return filename

def extract_text_from_pdf(pdf_path):
    """Real extraction of text from a PDF file."""
    print(f"   -> (Tool: Reading real PDF '{pdf_path}'...)")
    try:
        reader = PdfReader(pdf_path)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text() + "\n"
        return full_text
    except Exception as e:
        return f"Error reading PDF: {e}"

def save_pdf_report(content, filename="Contract_Review_Report.pdf"):
    """Generates a professional PDF report with fixed formatting."""
    doc = SimpleDocTemplate(filename, pagesize=LETTER)
    styles = getSampleStyleSheet()
    story = []

    story.append(Paragraph("ContractGuard AI Report", styles['Title']))
    story.append(Spacer(1, 12))

    for line in content.split('\n'):
        line = line.strip()
        if not line:
            continue
            
        # 1. Handle Headers (###)
        if line.startswith("###"):
            header_text = line.replace('#', '').strip()
            story.append(Paragraph(header_text, styles['Heading2']))
            
        # 2. Handle Standard Text
        else:
            # Escape HTML special characters first to avoid XML parsing errors
            # (e.g., turning "Net < 30" into "Net &lt; 30")
            clean_line = line.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
            
            # Use Regex to correctly swap **bold** with <b>bold</b>
            clean_line = re.sub(r'\*\*(.*?)\*\*', r'<b>\1</b>', clean_line)
            
            story.append(Paragraph(clean_line, styles['BodyText']))
            
        story.append(Spacer(1, 6))

    doc.build(story)
    print(f"✅ Saved professional report to: {filename}")
    return filename

In [10]:
# --- 4. MAIN WORKFLOW LOGIC ---

def run_contract_guard(pdf_path):
    print(f"🚀 STARTING ContractGuard Session ({MODEL_NAME})...")

    # A. DEFINE THE PLAYBOOK (The 'Brain' of the operation)
    PLAYBOOK = """
    POLICY RULES:
    1. Payment Terms: Must be Net-30 or Net-45. (Net-90 or higher is a VIOLATION).
    2. Liability Cap: Must be at least 1x Annual Fees. (Caps below $10,000 are a VIOLATION).
    3. Jurisdiction: Must be USA (Delaware, NY, or CA). (Antarctica or offshore is a VIOLATION).
    """

    # B. INITIALIZE AGENTS
    
    # Analyst: Finds the data
    analyst = SessionAgent("Analyst", 
        "You are a Senior Legal Analyst. Read the text, understand context, and extract clauses related to Payment, Liability, and Jurisdiction verbatim.")
    
    # Compliance: Checks the rules
    compliance = SessionAgent("Compliance", 
        f"You are a Strict Compliance Officer. Compare clauses against this policy:\n{PLAYBOOK}\n"
        "Output format per clause: Name -> Status (PASS/FAIL) -> Reason.")
    
    # Negotiator: Fixes the problems
    negotiator = SessionAgent("Negotiator", 
        "You are a Legal Negotiator. Rewrite violating clauses to be compliant and draft a polite email to the vendor.")

    # C. EXECUTE STEPS
    
    # 1. Ingest
    contract_text = extract_text_from_pdf(pdf_path)
    
    # 2. Analyze
    extracted_data = analyst.send_message(f"Extract key clauses from this text:\n{contract_text}")
    print(f"\n--- ANALYST REPORT ---\n{extracted_data}")

    # 3. Compliance Check
    compliance_report = compliance.send_message(f"Check these clauses against policy:\n{extracted_data}")
    print(f"\n--- COMPLIANCE REPORT ---\n{compliance_report}")

    # 4. Negotiate (only if needed)
    final_output = ""
    if "VIOLATION" in compliance_report.upper() or "FAIL" in compliance_report.upper():
        print("\n⚠️ Violations found. Drafting Redlines...")
        final_output = negotiator.send_message(
            f"Based on this report:\n{compliance_report}\n\n"
            "1. Provide redlined clauses.\n2. Draft an email to the vendor."
        )
        print(f"\n--- NEGOTIATION DRAFT ---\n{final_output}")
    else:
        final_output = "Contract Approved. No changes needed."
        print("\n✅ Contract Approved!")

    # 5. Save Report
    full_report = f"EXTRACTED CLAUSES\n{extracted_data}\n\nCOMPLIANCE FINDINGS\n{compliance_report}\n\nNEGOTIATION STRATEGY\n{final_output}"
    save_pdf_report(full_report)

In [11]:
if __name__ == "__main__":
    # Create a dummy file to test the system
    test_file = create_test_pdf("vendor_contract.pdf")
    
    # Run the AI on the file
    run_contract_guard(test_file)

✅ Created test file: vendor_contract.pdf
🚀 STARTING ContractGuard Session (gemini-2.5-flash)...
   -> (Tool: Reading real PDF 'vendor_contract.pdf'...)

🤖 Analyst is thinking...

--- ANALYST REPORT ---
Here are the extracted key clauses:

*   **Payment:** "Client shall pay all undisputed invoices within Net 90 Days."
*   **Liability:** "Vendor's total liability shall not exceed $100 USD."
*   **Jurisdiction:** "This agreement is governed by the laws of Antarctica."

🤖 Compliance is thinking...

--- COMPLIANCE REPORT ---
**Payment** -> **FAIL** -> Net-90 is a violation of Policy Rule 1, which requires Net-30 or Net-45.

**Liability** -> **FAIL** -> The liability cap of $100 USD is below the minimum allowed cap of $10,000, which is a violation of Policy Rule 2.

**Jurisdiction** -> **FAIL** -> Antarctica is explicitly listed as a violation for jurisdiction under Policy Rule 3.

⚠️ Violations found. Drafting Redlines...

🤖 Negotiator is thinking...

--- NEGOTIATION DRAFT ---
Okay, as your